#Heuristic Search Techniques

### ✅ 1. Water Jug Problem using Depth First Search

In [6]:
def dfs(jug1, jug2, goal):
    stack, visited = [(0, 0)], set()

    while stack:
        a, b = stack.pop()
        if (a, b) in visited: continue
        visited.add((a, b))
        print(a, b)
        if a == goal or b == goal: return True

        stack += [
            (jug1, b), (a, jug2), (0, b), (a, 0),
            (min(jug1, a + b), b - (min(jug1, a + b) - a)),
            (a - (min(jug2, a + b) - b), min(jug2, a + b))
        ]
    return False

dfs(4, 3, 2)

0 0
0 3
3 0
3 3
4 2


True

### ✅ 2. TSP – Branch and Bound

In [7]:
from itertools import permutations

def tsp_brute(cost):
    n, best = len(cost), float('inf')
    for path in permutations(range(n)):
        total = sum(cost[path[i]][path[(i+1)%n]] for i in range(n))
        best = min(best, total)
    print("Min Cost:", best)

tsp_brute([[0,10,15,20],[10,0,35,25],[15,35,0,30],[20,25,30,0]])

Min Cost: 80


### ✅ 3. TSP – Nearest Neighbour (Simplified)

In [8]:
def tsp_nn(graph, start=0):
    n, visited, path, cost = len(graph), [False]*len(graph), [start], 0
    visited[start] = True
    for _ in range(n-1):
        cur = path[-1]
        next_city = min([(i, graph[cur][i]) for i in range(n) if not visited[i]], key=lambda x:x[1])
        path.append(next_city[0])
        visited[next_city[0]] = True
        cost += next_city[1]
    cost += graph[path[-1]][start]
    print("Path:", path + [start], "| Cost:", cost)

tsp_nn([[0,10,15,20],[10,0,35,25],[15,35,0,30],[20,25,30,0]])

Path: [0, 1, 3, 2, 0] | Cost: 80


### ✅ 4. Hill Climbing – Simple Version

In [9]:
def hill_climb(f, start):
    x = start
    while f(x+1) > f(x): x += 1
    print("Best x:", x, "| f(x):", f(x))

hill_climb(lambda x: -x*x + 5, 0)

Best x: 0 | f(x): 5


### ✅ 5. Best First Search – Simplified

In [10]:
import heapq

def best_first(graph, start, goal, h):
    q, visited = [(h[start], start)], set()
    while q:
        _, node = heapq.heappop(q)
        print("Visiting:", node)
        if node == goal: return
        visited.add(node)
        for n in graph[node]:
            if n not in visited:
                heapq.heappush(q, (h[n], n))

g = {'A':['B','C'],'B':['D','E'],'C':['F'],'D':[],'E':['F'],'F':[]}
h = {'A':5,'B':4,'C':3,'D':2,'E':1,'F':0}
best_first(g, 'A', 'F', h)

Visiting: A
Visiting: C
Visiting: F


### ✅ 6. A* Algorithm – Pathfinding with Intelligence

In [25]:
from queue import PriorityQueue

def a_star(start, goal, graph, h):
    open_set = PriorityQueue()
    open_set.put((0, start))
    came_from = {}
    g = {node: float('inf') for node in graph}
    g[start] = 0

    while not open_set.empty():
        _, current = open_set.get()

        if current == goal:
            path = []
            while current in came_from:
                path.append(current)
                current = came_from[current]
            path.append(start)
            return path[::-1]

        for neighbor, cost in graph[current]:
            tentative_g = g[current] + cost
            if tentative_g < g[neighbor]:
                came_from[neighbor] = current
                g[neighbor] = tentative_g
                f = tentative_g + h[neighbor]
                open_set.put((f, neighbor))

    return None

graph = {
    'A': [('B', 1), ('C', 4)],
    'B': [('D', 5), ('E', 12)],
    'C': [('F', 2)],
    'D': [('G', 3)],
    'E': [('G', 1)],
    'F': [('G', 5)],
    'G': []
}

h = {  # Heuristics
    'A': 7, 'B': 6, 'C': 2, 'D': 1,
    'E': 5, 'F': 3, 'G': 0
}

path = a_star('A', 'G', graph, h)
print("Shortest path:", path)


Shortest path: ['A', 'B', 'D', 'G']


### ✅ 7. AO* Algorithm – AND/OR Graph Search

In [26]:
graph = {
    'A': [('B', 'C')],
    'B': [('D',)],
    'C': [('E', 'F')],
    'D': [],
    'E': [],
    'F': []
}

heuristic = {'A': 10, 'B': 4, 'C': 2, 'D': 0, 'E': 0, 'F': 0}
solved = {}

def ao_star(node):
    if node in solved:
        return heuristic[node]

    children = graph.get(node, [])
    if not children:
        solved[node] = True
        return heuristic[node]

    min_cost = float('inf')
    best = None

    for group in children:
        cost = sum(ao_star(child) for child in group)
        if cost < min_cost:
            min_cost = cost
            best = group

    heuristic[node] = min_cost
    solved[node] = True
    print(f"{node} -> {best} [Cost: {min_cost}]")
    return min_cost

ao_star('A')


B -> ('D',) [Cost: 0]
C -> ('E', 'F') [Cost: 0]
A -> ('B', 'C') [Cost: 0]


0

# Constraint Satisfaction Problem

### ✅ 1. Tic-Tac-Toe using CSP (Constraint Satisfaction Problem)

In [12]:
def check_win(board, player):
    wins = [(0,1,2),(3,4,5),(6,7,8), (0,3,6),(1,4,7),(2,5,8), (0,4,8),(2,4,6)]
    return any(all(board[i]==player for i in line) for line in wins)

def csp_tictactoe():
    board = [' '] * 9
    for turn in range(9):
        player = 'X' if turn % 2 == 0 else 'O'
        for i in range(9):
            if board[i] == ' ':
                board[i] = player
                if check_win(board, player):
                    print(f"{player} wins!\n{board}")
                    return
                break
    print("Draw:", board)

csp_tictactoe()

X wins!
['X', 'O', 'X', 'O', 'X', 'O', 'X', ' ', ' ']


### ✅ 2. Sudoku using CSP

In [13]:
def is_valid(board, row, col, num):
    for i in range(9):
        if board[row][i] == num or board[i][col] == num:
            return False
    box = 3 * (row//3), 3 * (col//3)
    for i in range(box[0], box[0]+3):
        for j in range(box[1], box[1]+3):
            if board[i][j] == num:
                return False
    return True

def solve_sudoku(board):
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0:
                for num in range(1, 10):
                    if is_valid(board, i, j, num):
                        board[i][j] = num
                        if solve_sudoku(board): return True
                        board[i][j] = 0
                return False
    return True

sudoku = [
    [5,3,0, 0,7,0, 0,0,0],
    [6,0,0, 1,9,5, 0,0,0],
    [0,9,8, 0,0,0, 0,6,0],
    [8,0,0, 0,6,0, 0,0,3],
    [4,0,0, 8,0,3, 0,0,1],
    [7,0,0, 0,2,0, 0,0,6],
    [0,6,0, 0,0,0, 2,8,0],
    [0,0,0, 4,1,9, 0,0,5],
    [0,0,0, 0,8,0, 0,7,9],
]
solve_sudoku(sudoku)
for row in sudoku: print(row)

[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


### ✅ 3. Tic-Tac-Toe using CAP (Constraint and Action Propagation)

In [14]:
def cap_tictactoe():
    board = [' '] * 9
    rules = lambda turn: 'X' if turn % 2 == 0 else 'O'

    for turn in range(9):
        player = rules(turn)
        for i in range(9):
            if board[i] == ' ':
                board[i] = player
                break
    print("Final board:", board)

cap_tictactoe()

Final board: ['X', 'O', 'X', 'O', 'X', 'O', 'X', 'O', 'X']


### ✅ 4. Sudoku using CAP (Simplified Logic Propagation)

In [15]:
def cap_sudoku(board):
    possibilities = [[[1,2,3,4,5,6,7,8,9] for _ in range(9)] for _ in range(9)]

    def eliminate():
        for i in range(9):
            for j in range(9):
                if board[i][j]:
                    val = board[i][j]
                    for k in range(9):
                        if val in possibilities[i][k]: possibilities[i][k].remove(val)
                        if val in possibilities[k][j]: possibilities[k][j].remove(val)

    eliminate()
    for i in range(9):
        for j in range(9):
            if board[i][j] == 0 and len(possibilities[i][j]) == 1:
                board[i][j] = possibilities[i][j][0]
    for row in board: print(row)

board = [
    [5,3,0,0,7,0,0,0,0],
    [6,0,0,1,9,5,0,0,0],
    [0,9,8,0,0,0,0,6,0],
    [8,0,0,0,6,0,0,0,3],
    [4,0,0,8,0,3,0,0,1],
    [7,0,0,0,2,0,0,0,6],
    [0,6,0,0,0,0,2,8,0],
    [0,0,0,4,1,9,0,0,5],
    [0,0,0,0,8,0,0,7,9],
]

cap_sudoku(board)

[5, 3, 0, 0, 7, 0, 0, 0, 0]
[6, 0, 0, 1, 9, 5, 0, 0, 0]
[0, 9, 8, 0, 0, 0, 0, 6, 0]
[8, 0, 0, 0, 6, 0, 0, 0, 3]
[4, 0, 0, 8, 5, 3, 0, 0, 1]
[7, 0, 0, 0, 2, 0, 0, 0, 6]
[0, 6, 0, 0, 0, 0, 2, 8, 0]
[0, 0, 0, 4, 1, 9, 0, 0, 5]
[0, 0, 0, 0, 8, 0, 0, 7, 9]


# Alpha-Beta Pruning (Simplified MiniMax with Pruning)

In [16]:
def alphabeta(depth, nodeIndex, isMax, values, alpha, beta):
    if depth == 3:
        return values[nodeIndex]

    if isMax:
        best = float('-inf')
        for i in range(2):
            val = alphabeta(depth+1, nodeIndex*2+i, False, values, alpha, beta)
            best = max(best, val)
            alpha = max(alpha, best)
            if beta <= alpha:
                break  # Beta Cut-off
        return best
    else:
        best = float('inf')
        for i in range(2):
            val = alphabeta(depth+1, nodeIndex*2+i, True, values, alpha, beta)
            best = min(best, val)
            beta = min(beta, best)
            if beta <= alpha:
                break  # Alpha Cut-off
        return best

# Simulated tree values (leaf nodes at depth 3)
values = [3, 5, 6, 9, 1, 2, 0, -1]
print("Optimal value:", alphabeta(0, 0, True, values, float('-inf'), float('inf')))

Optimal value: 5


In [ ]:
         MAX
        /    \
      MIN    MIN
     /  \    /  \
   MAX MAX  MAX MAX
   / \  / \  / \  / \
  3  5 6  9 1 2 0  -1


# 8 Puzzle Problem using A* Search (Shortest Path Finder)

In [17]:
import heapq

goal = [[1,2,3],[4,5,6],[7,8,0]]

def h(state):  # Heuristic: Number of misplaced tiles
    return sum(state[i][j] != goal[i][j] and state[i][j] != 0 for i in range(3) for j in range(3))

def flatten(state): return tuple(num for row in state for num in row)

def get_moves(x, y):
    return [(x+dx, y+dy) for dx,dy in [(-1,0),(1,0),(0,-1),(0,1)] if 0 <= x+dx < 3 and 0 <= y+dy < 3]

def a_star(start):
    open_set = []
    heapq.heappush(open_set, (h(start), 0, start))
    visited = set()

    while open_set:
        _, cost, state = heapq.heappop(open_set)
        if state == goal:
            print("Solved with cost:", cost)
            for row in state: print(row)
            return

        visited.add(flatten(state))
        x, y = next((i,j) for i in range(3) for j in range(3) if state[i][j] == 0)

        for nx, ny in get_moves(x, y):
            new_state = [row[:] for row in state]
            new_state[x][y], new_state[nx][ny] = new_state[nx][ny], new_state[x][y]

            if flatten(new_state) not in visited:
                heapq.heappush(open_set, (cost + 1 + h(new_state), cost + 1, new_state))

start_state = [
    [1, 2, 3],
    [4, 0, 6],
    [7, 5, 8]
]

a_star(start_state)


Solved with cost: 2
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]


# 8 Queen Problem (Backtracking)

In [18]:
def is_safe(board, row, col):
    for i in range(row):
        if board[i] == col or \
           abs(board[i] - col) == abs(i - row):  # diagonal check
            return False
    return True

def solve_queens(row, board, solutions):
    if row == 8:
        solutions.append(board[:])
        return
    for col in range(8):
        if is_safe(board, row, col):
            board[row] = col
            solve_queens(row + 1, board, solutions)

def print_board(board):
    for i in board:
        print('. ' * i + 'Q ' + '. ' * (7 - i))
    print('\n' + '='*20 + '\n')

solutions = []
solve_queens(0, [0]*8, solutions)
print(f"Total solutions: {len(solutions)}")
print_board(solutions[0])  # Print one solution


Total solutions: 92
Q . . . . . . . 
. . . . Q . . . 
. . . . . . . Q 
. . . . . Q . . 
. . Q . . . . . 
. . . . . . Q . 
. Q . . . . . . 
. . . Q . . . . 




# Genetic Algorithm

### ✅ Genetic Algorithm – Simplified Version

In [19]:
import random

# Convert binary string to integer
def binary_to_int(b):
    return int(b, 2)

# Fitness function: f(x) = x^2
def fitness(b):
    x = binary_to_int(b)
    return x * x

# Selection: pick 2 best from population
def select(pop):
    return sorted(pop, key=fitness, reverse=True)[:2]

# Crossover: combine parents
def crossover(p1, p2):
    point = random.randint(1, len(p1)-2)
    return p1[:point] + p2[point:], p2[:point] + p1[point:]

# Mutation: randomly flip one bit
def mutate(b):
    i = random.randint(0, len(b)-1)
    b = list(b)
    b[i] = '1' if b[i] == '0' else '0'
    return ''.join(b)

# GA main loop
def genetic_algorithm():
    population = [''.join(random.choice('01') for _ in range(5)) for _ in range(4)]

    for gen in range(10):
        print(f"\nGeneration {gen}:", population)
        selected = select(population)
        print("Selected:", selected)
        child1, child2 = crossover(*selected)
        child1, child2 = mutate(child1), mutate(child2)
        population = selected + [child1, child2]

    best = max(population, key=fitness)
    print("\nBest solution:", best, "→ x =", binary_to_int(best), "→ f(x) =", fitness(best))

genetic_algorithm()



Generation 0: ['01010', '10010', '01111', '00110']
Selected: ['10010', '01111']

Generation 1: ['10010', '01111', '10011', '01011']
Selected: ['10011', '10010']

Generation 2: ['10011', '10010', '10000', '10010']
Selected: ['10011', '10010']

Generation 3: ['10011', '10010', '10011', '10010']
Selected: ['10011', '10011']

Generation 4: ['10011', '10011', '11011', '10001']
Selected: ['11011', '10011']

Generation 5: ['11011', '10011', '11010', '10111']
Selected: ['11011', '11010']

Generation 6: ['11011', '11010', '10010', '11001']
Selected: ['11011', '11010']

Generation 7: ['11011', '11010', '11011', '11001']
Selected: ['11011', '11011']

Generation 8: ['11011', '11011', '11111', '10011']
Selected: ['11111', '11011']

Generation 9: ['11111', '11011', '11010', '01111']
Selected: ['11111', '11011']

Best solution: 11111 → x = 31 → f(x) = 961


### ✅ Case Study: Electricity Estimation using Genetic Algorithm

In [20]:
import random

# Appliance data: (power in kW, [availability per hour])
appliances = {
    'AC':    (1.5, [1]*24),
    'Heater':(2.0, [0]*6 + [1]*6 + [0]*12),
    'Fridge':(0.8, [1]*24),
    'Washer':(1.0, [0]*8 + [1]*4 + [0]*12),
}

# Tariff per hour (randomized for simplicity)
tariff = [5 if 8<=i<=18 else 3 for i in range(24)]

# Generate random schedule: each appliance has on/off for 24 hours
def random_schedule():
    return {appl: [random.randint(0, 1) if appliances[appl][1][h] else 0 for h in range(24)] for appl in appliances}

# Fitness = total electricity cost
def fitness(schedule):
    total_cost = 0
    for h in range(24):
        hour_load = sum(schedule[a][h] * appliances[a][0] for a in appliances)
        total_cost += hour_load * tariff[h]
    return -total_cost  # Minimize cost → higher fitness = less cost

# Selection
def select(pop):
    return sorted(pop, key=fitness, reverse=True)[:2]

# Crossover: take half from each parent
def crossover(p1, p2):
    child = {}
    for a in appliances:
        child[a] = [random.choice([p1[a][i], p2[a][i]]) for i in range(24)]
    return child

# Mutation: flip one bit in a random appliance at random hour
def mutate(schedule):
    a = random.choice(list(appliances))
    h = random.randint(0, 23)
    if appliances[a][1][h] == 1:
        schedule[a][h] ^= 1  # flip bit
    return schedule

# Main GA
def ga():
    population = [random_schedule() for _ in range(6)]

    for gen in range(10):
        selected = select(population)
        child1 = mutate(crossover(*selected))
        child2 = mutate(crossover(*selected))
        population = selected + [child1, child2]

    best = max(population, key=fitness)
    print("\nBest Schedule Found (Cost: ₹", -fitness(best), ")")
    for a in best:
        print(f"{a:>7}: {''.join(str(x) for x in best[a])}")

ga()



Best Schedule Found (Cost: ₹ 83.9 )
     AC: 100101001000001110110000
 Heater: 000000010000000000000000
 Fridge: 010110100101000000000011
 Washer: 000000000000000000000000


# Fuzzy Logic

### ✅ Fuzzy Logic – Basics

In [21]:
def fuzzy_temp(temp):
    cold = max(0, min(1, (20 - temp) / 10))
    warm = max(0, min((temp - 15) / 10, (30 - temp) / 10))
    hot = max(0, min(1, (temp - 25) / 10))
    return cold, warm, hot

def infer_fan_speed(cold, warm, hot):
    # Rule base:
    # IF Cold → Low (30), Warm → Medium (60), Hot → High (90)
    low = cold * 30
    medium = warm * 60
    high = hot * 90

    total_weight = cold + warm + hot
    if total_weight == 0:
        return 0  # default case
    return (low + medium + high) / total_weight

# Example usage
temp = 27
cold, warm, hot = fuzzy_temp(temp)
speed = infer_fan_speed(cold, warm, hot)

print(f"Temp: {temp}°C → Cold={cold:.2f}, Warm={warm:.2f}, Hot={hot:.2f}")
print(f"Fan Speed: {speed:.2f}%")


Temp: 27°C → Cold=0.00, Warm=0.30, Hot=0.20
Fan Speed: 72.00%


### ✅ Case Study: Fuzzy Logic Controller for Peak Load Shaving

In [22]:
def fuzzify_load(load):
    low = max(0, min(1, (40 - load) / 20))
    medium = max(0, min((load - 30)/20, (70 - load)/20))
    high = max(0, min(1, (load - 60) / 20))
    return low, medium, high

def fuzzify_time(hour):
    if hour < 8 or hour >= 22:
        return 1, 0, 0  # Off-peak
    elif hour < 17:
        return 0, 1, 0  # Mid-peak
    else:
        return 0, 0, 1  # Peak

def infer_action(load_vals, time_vals):
    low, med, high = load_vals
    off, mid, peak = time_vals

    # Fuzzy rules:
    # If Load is HIGH and Time is PEAK → Full Action
    # If Load is MED and Time is MID → Moderate
    # Else → No Action or light action

    no_act = low * off
    mod_act = med * mid
    full_act = high * peak

    total = no_act + mod_act + full_act
    if total == 0:
        return 0  # Default: no action

    action = (no_act * 0 + mod_act * 50 + full_act * 100) / total
    return action

# Example
load = 75  # in %
hour = 18  # 6 PM → Peak time

load_vals = fuzzify_load(load)
time_vals = fuzzify_time(hour)

action = infer_action(load_vals, time_vals)

print(f"Load: {load}%, Time: {hour}:00")
print(f"Suggested Load Reduction: {action:.2f}%")


Load: 75%, Time: 18:00
Suggested Load Reduction: 100.00%
